In [1]:
import torch
from torch.autograd import Function
from torch.autograd import Variable as V
import torch as t
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
device = torch.device("cuda")
device1 = torch.device("cpu")
import torch.optim as optim
import time
import pandas as pd
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules import Module
from torch.nn.modules.lazy import LazyModuleMixin
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
#Mean Square Error
def MSE(pred,true):
    return np.mean((pred-true)**2)

#Root Mean Square Error
def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

#Mean Absolute Error
def MAE(pred, true):
    return np.mean(np.abs(pred-true))

#Mean Absolute Percentage Error
def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

#Relative Squared Error
def RSE(pred, true):
    return np.sqrt(np.sum((true-pred)**2)) / np.sqrt(np.sum((true-true.mean())**2))

def CORR(pred, true):
    u = ((true-true.mean(0))*(pred-pred.mean(0))).sum(0) 
    d = np.sqrt(((true-true.mean(0))**2*(pred-pred.mean(0))**2).sum(0))
    return (u/d).mean(-1)

'''
def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))
'''

#Coefficient of Determination
def R2(pred, true):
    return r2_score(pred,true) 


def gamma(a):
    
    if a <= 0 and a %1 == 0:
        return float('inf')
    
    return math.gamma(a)


#This is not an important Class; its purpose is solely to obtain the weight matrix W and bias b.
#We have rearranged the positions of 'in_features' and 'out_features' for ease of future use.
class Linear(Module):

    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((in_features, out_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight.T, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
    
    
def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test


#fogd_MSELoss
class MSELoss_fractional(Function):
    
    def forward(ctx,input1,target):
        loss = nn.MSELoss()
        ctx.save_for_backward(input1,target)
        return loss(input1,target)
    
    def backward(ctx,grad_outputs):
        input1,target = ctx.saved_tensors
        y_pred_f = t.abs(input1 - target)
        grad_output = 2*y_pred_f**(2-a)/gamma(3-a)
        return grad_output,None
    
#fogd_Linear
class Linear_fractional(Function):
    
    def forward(ctx,input1,w,b):
        ctx.save_for_backward(input1,w,b)
        outputs = input1@w + b
        return outputs
    
    def backward(ctx,grad_outputs):
        input1,w,b = ctx.saved_tensors
        
        x_grad = torch.mm(grad_outputs,w.T)
        w_grad = torch.mm(input1.T,grad_outputs)
        b_grad = torch.mul(grad_outputs,b)
        return x_grad,w_grad,b_grad
    

In [8]:
#GSPC，DJI，IXIC，FTSE，GDAXI，N225 data preprocessing
time1 = time.time()
slide_windows_size = 36  #i.e.,input length
stock_array = ['GSPC','DJI','IXIC','FTSE','GDAXI','N225']   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
#stock_array = ['ETTh1','ETTh2','ETTm1','ETTm2']   #ETTh1，ETTh2，ETTm1，ETTm2
for stock in stock_array:
    #stock = 'DJI'   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
    pre_days = 48     #i.e.,prediction lengths,48,96,192,384
    while pre_days <= 384:  
        df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
        del df_DJIA['Date']
        #del df_DJIA['date']        #ETT
        sca_DJIA = scaler.fit_transform(df_DJIA)
        label = t.tensor([row[4] for row in sca_DJIA][slide_windows_size:]).view(-1,1)
        #label = t.tensor([row[6] for row in sca_DJIA][slide_windows_size:]).view(-1,1)  #ETT
        sca_DJIA = t.tensor(sca_DJIA[:-pre_days])
        #label
        deq_DJIA = deque(maxlen=pre_days)
        DJIA_label = []
        for i in label:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==pre_days:
                DJIA_label.append(list(deq_DJIA))
        DJIA_label = t.Tensor(DJIA_label).view(-1,pre_days)
        #X
        deq_DJIA = deque(maxlen=slide_windows_size)
        DJIA_X = []
        for i in sca_DJIA:
            deq_DJIA.append(list(i))
            if len(deq_DJIA)==slide_windows_size:
                DJIA_X.append(list(deq_DJIA))
        DJIA_X = t.Tensor(DJIA_X)
        X_DJIA, y_DJIA = DJIA_X.to(device), DJIA_label.to(device)
        X_train,X_val,X_test,y_train,y_val,y_test = split(X_DJIA,y_DJIA)   #7:2:1

        class Net(nn.Module):
            def __init__(self,):
                super(Net,self).__init__()
                self.linear1 = Linear(slide_windows_size*5,128)
                #self.linear1 = Linear(slide_windows_size*7,128)  #ETT
                self.linear2 = Linear(128,64)
                self.linear3 = Linear(64,32)
                self.linear4 = Linear(32,pre_days)
                self.w1,self.b1 = self.linear1.weight,self.linear1.bias
                self.w2,self.b2 = self.linear2.weight,self.linear2.bias
                self.w3,self.b3 = self.linear3.weight,self.linear3.bias
                self.w4,self.b4 = self.linear4.weight,self.linear4.bias

            def forward(self,x):
                x = x.view(-1,slide_windows_size*5)
                #x = x.view(-1,slide_windows_size*7)   #ETT
                y2 = Linear_fractional.apply(x,self.w1,self.b1)   
                y3 = Linear_fractional.apply(y2,self.w2,self.b2)   
                y4 = Linear_fractional.apply(y3,self.w3,self.b3) 
                y5 = Linear_fractional.apply(y4,self.w4,self.b4)  
                return y5

        net = Net()
        torch.save(net.state_dict(), r'./model/Adam_Stock_FOGD/'+stock+'_'+str(pre_days)+'_net.pth')    

        a = 0.6
        lr =5e-5   #adam:5e-5
        epoch = 1000
        batch_size = 256
        train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11 = [],[],[],[],[]  #collecting train set loss
        while a < 1.1:
            a = round(a + 0.1,1)
            best_loss = 10
            net = Net()
            net.load_state_dict(torch.load(r'./model/Adam_Stock_FOGD/'+stock+'_'+str(pre_days)+'_net.pth'))
            optimizer = torch.optim.Adam(net.parameters(),lr=lr)
            net.to(device)
            for i in range(epoch):
                loss_sum = 0
                net.train()  
                start = np.random.randint(11,size=1)[0]  #Generate a random integer between 0 and 10
                while start+batch_size<len(X_train):
                    X_batch = X_train[start:start+batch_size]
                    y_batch = y_train[start:start+batch_size]
                    start = start+batch_size
                    out = net(X_batch)
                    loss = MSELoss_fractional.apply(out,y_batch) 
                    loss.backward()
                    loss_sum += loss
                    optimizer.step()
                    net.zero_grad()
                if a==0.7:
                    train_loss_07.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.8:
                    train_loss_08.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==0.9:
                    train_loss_09.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.0:
                    train_loss_10.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                if a==1.1:
                    train_loss_11.append(loss_sum.cpu().detach().numpy())
                    if (i+1)%50==0:
                        print(loss_sum.cpu().detach().numpy())
                net.eval()
                with torch.no_grad():
                    out = net(X_val)
                    MSE_val = MSE(y_val.cpu().detach().numpy(),out.cpu().detach().numpy())   #MSE，MAPE
                    if best_loss > MSE_val:
                        best_loss = MSE_val
                        torch.save(net.state_dict(), r'./model/Adam_Stock_FOGD/'+stock+'_net_fractional_'+str(a).replace(".", "_")+'_'+str(pre_days)+'.pth')
                        print('epoch:{0}'.format(i+1)+'\n'+'Validation Loss:{0}'.format(MSE_val))
            print('the order'+str(a)+'finished!')
        if pre_days==48:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adam_Stock_FOGD/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==96:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adam_Stock_FOGD/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==192:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adam_Stock_FOGD/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        if pre_days==384:
            data = pd.DataFrame([train_loss_07,train_loss_08,train_loss_09,train_loss_10,train_loss_11],
                                index=['train_loss_07','train_loss_08','train_loss_09','train_loss_10','train_loss_11'])
            data.to_csv(r'./train_loss_data/Adam_Stock_FOGD/'+stock+'_net_fractional_'+str(pre_days)+'.csv')
            print(stock+' about '+str(pre_days)+' finished!')
        pre_days = pre_days*2
time2 = time.time()
print(time2 - time1)

epoch:1
Validation Loss:0.9169321656227112
epoch:2
Validation Loss:0.8473507165908813
epoch:3
Validation Loss:0.7840248346328735
epoch:4
Validation Loss:0.7221591472625732
epoch:5
Validation Loss:0.6599406599998474
epoch:6
Validation Loss:0.597151517868042
epoch:7
Validation Loss:0.5343833565711975
epoch:8
Validation Loss:0.4732467234134674
epoch:9
Validation Loss:0.4159325063228607
epoch:10
Validation Loss:0.36683040857315063
epoch:11
Validation Loss:0.3401069939136505
50973284.0
19207582000.0
371427340000.0
2546258100000.0
10453962000000.0
31822262000000.0
79537220000000.0
173979130000000.0
343544540000000.0
630057200000000.0
1090148460000000.0
1796093700000000.0
2839011800000000.0
4336631000000000.0
6446557000000000.0
9345562000000000.0
1.3289751e+16
1.8494565e+16
2.532691e+16
3.408765e+16
the order0.7finished!
epoch:1
Validation Loss:0.9149428009986877
epoch:2
Validation Loss:0.8438627123832703
epoch:3
Validation Loss:0.7789604067802429
epoch:4
Validation Loss:0.715470016002655
epo

332765630000.0
2311741500000.0
9573527000000.0
29270972000000.0
73617550000000.0
160878520000000.0
319187880000000.0
584611300000000.0
1011146200000000.0
1662729200000000.0
2626184300000000.0
4012364400000000.0
5952124600000000.0
8616490000000000.0
1.2203358e+16
1.6503597e+16
2.0692633e+16
2.548943e+16
the order0.7finished!
epoch:1
Validation Loss:0.7987220287322998
epoch:2
Validation Loss:0.7837705016136169
epoch:3
Validation Loss:0.765802800655365
epoch:4
Validation Loss:0.7410392761230469
epoch:5
Validation Loss:0.7068647146224976
epoch:6
Validation Loss:0.6618570685386658
epoch:7
Validation Loss:0.6064276099205017
epoch:8
Validation Loss:0.542870044708252
epoch:9
Validation Loss:0.47443145513534546
epoch:10
Validation Loss:0.40480297803878784
epoch:11
Validation Loss:0.34271159768104553
epoch:12
Validation Loss:0.3293578624725342
46626216.0
16306552000.0
315215970000.0
2179734000000.0
9039850000000.0
27686466000000.0
69577820000000.0
152885900000000.0
303071480000000.0
557270430000

515979820000000.0
945389900000000.0
1630630900000000.0
2685673700000000.0
4240480500000000.0
6498053600000000.0
9653424000000000.0
1.3991754e+16
1.9847667e+16
2.7639697e+16
3.7738074e+16
5.092413e+16
the order0.7finished!
epoch:1
Validation Loss:1.0617765188217163
epoch:2
Validation Loss:0.9798189997673035
epoch:3
Validation Loss:0.8935312032699585
epoch:4
Validation Loss:0.7989677786827087
epoch:5
Validation Loss:0.6968956589698792
epoch:6
Validation Loss:0.5920881628990173
epoch:7
Validation Loss:0.49348074197769165
epoch:8
Validation Loss:0.4196174442768097
91325336.0
28319050000.0
524204340000.0
3545455700000.0
14499345000000.0
44012052000000.0
109854720000000.0
240286640000000.0
476160400000000.0
875108900000000.0
1514919700000000.0
2499751000000000.0
3963259500000000.0
6069206000000000.0
9036365000000000.0
1.3101144e+16
1.8644253e+16
2.5999005e+16
3.5655534e+16
4.8203277e+16
the order0.8finished!
epoch:1
Validation Loss:1.0606908798217773
epoch:2
Validation Loss:0.977561891078949

12637800000000.0
38690982000000.0
97450600000000.0
213298130000000.0
423589370000000.0
778063500000000.0
1350777400000000.0
2222328300000000.0
3520322700000000.0
5387157000000000.0
8007521500000000.0
1.1624519e+16
1.6536291e+16
2.2980665e+16
3.1519713e+16
4.2476488e+16
the order0.8finished!
epoch:1
Validation Loss:1.0341765880584717
epoch:2
Validation Loss:1.001961588859558
epoch:3
Validation Loss:0.9578948616981506
epoch:4
Validation Loss:0.8984114527702332
epoch:5
Validation Loss:0.8225203156471252
epoch:6
Validation Loss:0.7331300973892212
epoch:7
Validation Loss:0.6375632882118225
epoch:8
Validation Loss:0.5496574640274048
epoch:9
Validation Loss:0.5011606812477112
70367590.0
21413570000.0
403733000000.0
2778244600000.0
11514453000000.0
35241580000000.0
88738930000000.0
195539610000000.0
388121360000000.0
716294240000000.0
1240587300000000.0
2054603100000000.0
3256446000000000.0
4999823500000000.0
7437715700000000.0
1.0810115e+16
1.5387597e+16
2.1473447e+16
2.9433465e+16
3.975498e+

531910730000000.0
979850750000000.0
1699464200000000.0
2809457800000000.0
4463089800000000.0
6852196000000000.0
1.0224876e+16
1.4884916e+16
2.1196883e+16
2.9620236e+16
4.068164e+16
5.5033705e+16
the order0.9finished!
epoch:1
Validation Loss:1.067018747329712
epoch:2
Validation Loss:0.9828736782073975
epoch:3
Validation Loss:0.9063590168952942
epoch:4
Validation Loss:0.8313233852386475
epoch:5
Validation Loss:0.7561986446380615
epoch:6
Validation Loss:0.6813309788703918
epoch:7
Validation Loss:0.6139695048332214
epoch:8
Validation Loss:0.5889747738838196
98794670.0
28586390000.0
523226500000.0
3532684800000.0
14465668000000.0
44076456000000.0
110804595000000.0
243550680000000.0
484983580000000.0
895173460000000.0
1557269100000000.0
2579769200000000.0
4106444300000000.0
6320444000000000.0
9450836000000000.0
1.37865e+16
1.9674852e+16
2.7538224e+16
3.7894746e+16
5.1343594e+16
the order1.0finished!
epoch:1
Validation Loss:1.0657812356948853
epoch:2
Validation Loss:0.9801223278045654
epoch:3

...

2705332500000000.0
the order0.9finished!
epoch:1
Validation Loss:1.1495195627212524
epoch:2
Validation Loss:1.1327545642852783
epoch:3
Validation Loss:1.1160776615142822
epoch:4
Validation Loss:1.096353530883789
epoch:5
Validation Loss:1.0712579488754272
epoch:6
Validation Loss:1.0389517545700073
epoch:7
Validation Loss:0.9980515241622925
epoch:8
Validation Loss:0.9477739334106445
epoch:9
Validation Loss:0.8888718485832214
epoch:10
Validation Loss:0.8232437968254089
epoch:11
Validation Loss:0.754218578338623
epoch:12
Validation Loss:0.6867882609367371
epoch:13
Validation Loss:0.6297173500061035
epoch:14
Validation Loss:0.6024494171142578
3227958.5
1154444700.0
22703706000.0
158845340000.0
665986400000.0
2059700900000.0
5234826400000.0
11560943000000.0
23165400000000.0
42833680000000.0
74562740000000.0
123674760000000.0
196961000000000.0
303356200000000.0
453673970000000.0
661865300000000.0
942442000000000.0
1320237200000000.0
1813903700000000.0
2456061200000000.0
the order1.0finished!


epoch:9
Validation Loss:0.4588283896446228
epoch:10
Validation Loss:0.411277711391449
19007948.0
5970250000.0
112004430000.0
764230700000.0
3141873000000.0
9589422000000.0
24116552000000.0
52972294000000.0
105386260000000.0
194340040000000.0
337485850000000.0
558352660000000.0
887573650000000.0
1363807400000000.0
2036853200000000.0
2965968900000000.0
4226092400000000.0
5905963000000000.0
8117298700000000.0
1.0981802e+16
the order0.9finished!
epoch:1
Validation Loss:1.0722705125808716
epoch:2
Validation Loss:0.9897235631942749
epoch:3
Validation Loss:0.9089909195899963
epoch:4
Validation Loss:0.8280872702598572
epoch:5
Validation Loss:0.74739009141922
epoch:6
Validation Loss:0.6676128506660461
epoch:7
Validation Loss:0.5897584557533264
epoch:8
Validation Loss:0.51558518409729
epoch:9
Validation Loss:0.45175087451934814
epoch:10
Validation Loss:0.42063048481941223
18744528.0
5550426600.0
102787875000.0
698845500000.0
2872247800000.0
8777826000000.0
22111955000000.0
48678937000000.0
97067

epoch:9
Validation Loss:0.6277347207069397
epoch:10
Validation Loss:0.5504499673843384
epoch:11
Validation Loss:0.496288925409317
21643958.0
7031592000.0
134858240000.0
938728500000.0
3896541200000.0
12001173000000.0
30359236000000.0
66876353000000.0
133185210000000.0
246113970000000.0
427136440000000.0
705583070000000.0
1120868700000000.0
1720050100000000.0
2566371000000000.0
3737068200000000.0
5314415400000000.0
7409318400000000.0
1.0180765e+16
1.3736255e+16
the order0.9finished!
epoch:1
Validation Loss:1.0597766637802124
epoch:2
Validation Loss:1.0338759422302246
epoch:3
Validation Loss:1.002459168434143
epoch:4
Validation Loss:0.9621411561965942
epoch:5
Validation Loss:0.9109683632850647
epoch:6
Validation Loss:0.8484050631523132
epoch:7
Validation Loss:0.7755185961723328
epoch:8
Validation Loss:0.6957250833511353
epoch:9
Validation Loss:0.6137279868125916
epoch:10
Validation Loss:0.539431631565094
epoch:11
Validation Loss:0.5062189102172852
21575300.0
6660265500.0
126385684000.0
8

epoch:10
Validation Loss:0.49973151087760925
epoch:11
Validation Loss:0.4620780050754547
5747418.0
2275519500.0
48488400000.0
358146240000.0
1545571000000.0
4884216000000.0
12573193000000.0
28063060000000.0
56439990000000.0
104529830000000.0
182340820000000.0
301753730000000.0
478303300000000.0
734463600000000.0
1095238600000000.0
1590231600000000.0
2257662200000000.0
3145138800000000.0
4305186600000000.0
5794048000000000.0
the order0.9finished!
epoch:1
Validation Loss:1.162499189376831
epoch:2
Validation Loss:1.0690828561782837
epoch:3
Validation Loss:0.9841020107269287
epoch:4
Validation Loss:0.9039453864097595
epoch:5
Validation Loss:0.8267861604690552
epoch:6
Validation Loss:0.752200186252594
epoch:7
Validation Loss:0.6802977323532104
epoch:8
Validation Loss:0.6110354661941528
epoch:9
Validation Loss:0.5454887747764587
epoch:10
Validation Loss:0.49030980467796326
epoch:11
Validation Loss:0.46496638655662537
5557323.5
2082846800.0
43516817000.0
319372430000.0
1379846000000.0
4370253

epoch:10
Validation Loss:0.5903439521789551
epoch:11
Validation Loss:0.5258623957633972
epoch:12
Validation Loss:0.46783018112182617
epoch:13
Validation Loss:0.433732807636261
5757836.5
2306979800.0
48869974000.0
359781240000.0
1555431600000.0
4905582500000.0
12613962000000.0
28101800000000.0
56466350000000.0
104478955000000.0
182312520000000.0
301824100000000.0
479493370000000.0
734878700000000.0
1094991200000000.0
1590504500000000.0
2253204500000000.0
3139670800000000.0
4298684600000000.0
5802626400000000.0
the order0.9finished!
epoch:1
Validation Loss:0.9499765038490295
epoch:2
Validation Loss:0.9313932061195374
epoch:3
Validation Loss:0.9107598066329956
epoch:4
Validation Loss:0.8856577277183533
epoch:5
Validation Loss:0.854184091091156
epoch:6
Validation Loss:0.814859926700592
epoch:7
Validation Loss:0.7670503258705139
epoch:8
Validation Loss:0.7111926674842834
epoch:9
Validation Loss:0.6490586996078491
epoch:10
Validation Loss:0.5836694240570068
epoch:11
Validation Loss:0.5195205

In [36]:
fogd__loss_48_DJI = pd.read_csv('train_loss_data/Adam_Stock_FOGD/DJI_net_fractional_48.csv')
ifogd__loss_48_DJI = pd.read_csv('train_loss_data/Adam_Stock/DJI_net_fractional_48.csv')

fogd__loss_48_FTSE = pd.read_csv('train_loss_data/Adam_Stock_FOGD/FTSE_net_fractional_48.csv')
ifogd__loss_48_FTSE = pd.read_csv('train_loss_data/Adam_Stock/FTSE_net_fractional_48.csv')

fogd__loss_48_GDAXI = pd.read_csv('train_loss_data/Adam_Stock_FOGD/GDAXI_net_fractional_48.csv')
ifogd__loss_48_GDAXI = pd.read_csv('train_loss_data/Adam_Stock/GDAXI_net_fractional_48.csv')

fogd__loss_48_GSPC = pd.read_csv('train_loss_data/Adam_Stock_FOGD/GSPC_net_fractional_48.csv')
ifogd__loss_48_GSPC = pd.read_csv('train_loss_data/Adam_Stock/GSPC_net_fractional_48.csv')

fogd__loss_48_IXIC = pd.read_csv('train_loss_data/Adam_Stock_FOGD/IXIC_net_fractional_48.csv')
ifogd__loss_48_IXIC = pd.read_csv('train_loss_data/Adam_Stock/IXIC_net_fractional_48.csv')

fogd__loss_48_N225 = pd.read_csv('train_loss_data/Adam_Stock_FOGD/N225_net_fractional_48.csv')
ifogd__loss_48_N225 = pd.read_csv('train_loss_data/Adam_Stock/N225_net_fractional_48.csv')

In [77]:
plt.figure(figsize=(10,6),dpi=1200)
custom_params = {
    'font.size': 10,
    'lines.linewidth': 1,
    'axes.labelsize': 10,
    'axes.titlesize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'grid.linestyle': '--',
    'grid.alpha': 0.7,
}

plt.rcParams.update(custom_params)

plt.plot(fogd__loss_48_DJI.iloc[2,1:1000],'r--',label=r'DJI')
plt.plot(fogd__loss_48_FTSE.iloc[2,1:1000],'b--',label=r'FTSE')
plt.plot(fogd__loss_48_GDAXI.iloc[2,1:1000],'y--',label=r'GDAXI')
plt.plot(fogd__loss_48_GSPC.iloc[2,1:1000],'g--',label=r'GSPC')
plt.plot(fogd__loss_48_IXIC.iloc[2,1:1000],'c--',label=r'IXIC')
plt.plot(fogd__loss_48_N225.iloc[2,1:1000],'k--',label=r'N225')

plt.plot(ifogd__loss_48_DJI.iloc[2,1:1000],'r-',label=r'DJI')
plt.plot(ifogd__loss_48_FTSE.iloc[2,1:1000],'b-',label=r'FTSE')
plt.plot(ifogd__loss_48_GDAXI.iloc[2,1:1000],'y-',label=r'GDAXI')
plt.plot(ifogd__loss_48_GSPC.iloc[2,1:1000],'g-',label=r'GSPC')
plt.plot(ifogd__loss_48_IXIC.iloc[2,1:1000],'c-',label=r'IXIC')
plt.plot(ifogd__loss_48_N225.iloc[2,1:1000],'k-',label=r'N225')

plt.xlim(0,100)
plt.xticks([0,10,20,30,40,50,60,70,80,90,100], fontsize=12, rotation=0)
plt.ylim(0,10)
plt.yticks([0,1,2,3,4,5,6,7,8,9,10], fontsize=12, rotation=0)
plt.legend()
plt.grid()

plt.xlabel(r'$epoch$',fontsize=15)
plt.ylabel(r'$loss$ of real datasets',fontsize=15)

plt.annotate(r'Dashed lines indicate the use of FOAdam',fontsize=12, xy=(15,8), xytext=(30,8.5),arrowprops=dict(facecolor='red'))
plt.annotate(r'Solid lines indicate the use of IFOAdam',fontsize=12, xy=(49,2.8), xytext=(55,3.5),arrowprops=dict(facecolor='red'))

plt.savefig('picture/Fig6.eps',bbox_inches='tight',format='eps')
plt.savefig('picture/Fig6.svg',bbox_inches='tight',format='svg')
plt.savefig('picture/Fig6.pdf',bbox_inches='tight',format='pdf')
plt.show()
plt.show()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
